In [ ]:
import pandas as pd

# disable logs for rdkit
from rdkit import RDLogger

RDLogger.DisableLog('rdApp.*')

random_seed = 42

from rdkit_mol_identifiers import (
  clean_structure_add_mol_id_columns,
  ensure_smiles_column,
)

import pandas_utils as pu


In [ ]:

def read_sdf(filename, progress_bar=False):
  smiles_list = []
  with open(filename, 'r', encoding='UTF-8') as file:
    for line in tqdm(file) if progress_bar else file:
      line = line.rstrip()
      if line == "> <SMILES>":
        smi = file.readline().rstrip()
        if smi:
          smiles_list.append(smi)

      # MoNA sdf computed SMILES=
      if line.startswith("computed SMILES="):
        smi = line[16:]
        if smi:
          smiles_list.append(smi)

      # NPAtlas SMILES format
      if '<compound_smiles>' in line:
        smi = file.readline().rstrip()
        if smi:
          smiles_list.append(smi)

    df = pd.DataFrame({
      "SMILES": smiles_list
    })
    return df


def read_msp_fast(filename, progress_bar=False):
  smiles_list = []
  with open(filename, 'r', encoding='UTF-8') as file:
    for line in tqdm(file) if progress_bar else file:
      key_value_pair = line.split(":")
      if len(key_value_pair) > 1 and key_value_pair[0].lower() == "smiles":
        smi = str(key_value_pair[1]).strip()
        if smi:
          smiles_list.append(smi)

    df = pd.DataFrame({
      "SMILES": smiles_list
    })
    return df



In [ ]:
df_mona_msms = read_sdf(
    r"C:\git\small_mol_database_review\databases\20231208\MoNA-export-LC-MS-MS_Spectra.sdf").rename(columns={"SMILES": "smiles"})
df_massbank_eu = read_msp_fast(
    r"C:\git\small_mol_database_review\databases\20231208\MassBank_NIST.msp").rename(columns={"SMILES": "smiles"})

In [ ]:
 df_mona_msms = clean_structure_add_mol_id_columns(df_mona_msms, drop_mol=True)

In [ ]:
df_mona_msms["inchikey"].nunique()

In [ ]:
pu.save_dataframe(df_mona_msms, r"C:\git\small_mol_database_review\databases\20231208\MoNA_LC_MSMS.tsv")

In [ ]:
df_massbank_eu = clean_structure_add_mol_id_columns(df_massbank_eu, drop_mol=True) 
df_massbank_eu

In [ ]:
pu.save_dataframe(df_massbank_eu, r"C:\git\small_mol_database_review\databases\20231208\MassBank_nist.tsv")

In [ ]:
df_massbank_eu["split_inchikey"].nunique()